## A. Configuraciones Generales.

In [1]:
#1. Librerías.
%run "../librerias.ipynb"

/home/ianlink95/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/ianlink95/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#2. Constantes.
#a. Constantes generales.
%run "../constantes.ipynb"

#b. Constantes a definir por el usuario.
#i. Dataset de lectura (post-feature engineering y undersampleado).
dataset_con_fe = dataset_file_fe_1_all_undersampleado
dataset_con_test = dataset_file_fe_1_all
#ii. Información sobre rango temporal del modelo.
cantidad_meses_train = "all"
ventana = 1
#iii. Meses de train y test.
mes_train = mes_train_all_menos_2_sin_rotas
mes_test = mes_test
#iv. Ruta de la BBDD donde se almacenan los hiperparámetros óptimos post-Optuna.
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = f"exp_lgbm_{cantidad_meses_train}_{ventana}_undersampling"

NameError: name 'dataset_file_fe_1_all' is not defined

In [3]:
#3. Lectura de datos.
#i. Train.
data = pd.read_parquet(dataset_con_fe)
#ii. Test.
test = pd.read_parquet(dataset_con_test)

NameError: name 'dataset_con_test' is not defined

In [ ]:
#4. Importo el estudio de Optuna.
study = optuna.load_study(
    study_name=study_name,
    storage=storage_name
    )

#### B. Pre-procesamiento.

In [ ]:
#1. Concateno Train y Test.
#i. Filtro el mes de interés en Test.
test = test[test["foto_mes"] == mes_test]
#ii. Borro la columna de clase binaria en train para poder concatenar.
data.drop(["clase_binaria"],axis=1,inplace=True)
#ii. Concateno.
data = pd.concat([data,test],axis=0)

In [ ]:
#2. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)

#ii. Pesos y reclusterización.
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#3. Dividimos entre conjuntos de datos.
#a. Datos para entrenar todo el modelo final para Kaggle.
train_data = data[data['foto_mes'].isin(mes_train)]
test_data = data[data['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria'], axis=1)
y_train_binaria = train_data['clase_binaria']
w_train = train_data['clase_peso']

#b. Datos de Test (a predecir).
X_test = test_data.drop(['clase_ternaria', 'clase_peso','clase_binaria'], axis=1)

## C. Semillerío.

In [ ]:
#1. Configuramos la cantidad de semillas.
cantidad_semillas = 100
semillas = [np.random.randint(0, 10000) for _ in range(cantidad_semillas)]

In [ ]:
#2. Construyo un DataFrame para guardar resultados.
resultados = pd.DataFrame()

In [ ]:
#3. Obtengo los mejores parámetros del estudio.
best_iter = study.best_trial.user_attrs["best_iter"]
best_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'verbose': 0  # La semilla se ajustará en cada iteración
}

In [ ]:
#4. Realizo 100 iteraciones con semillas distintas.
num_semilla = 1
for semilla in semillas:
    print("\n---------------------------------------")
    print("Semilla {}: {}.".format(num_semilla,semilla))
    num_semilla += 1
    #i. Listo los parámetros con la semilla correspondiente.
    print("--> Mejores parámetros...")
    best_params['seed'] = semilla
    #ii. Adaptamos el dataset de entrenamiento.
    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria,
                            weight=w_train)
    #iii. Entreno el modelo.
    print("--> Comienza el entrenamiento...")
    model_lgb = lgb.train(best_params,
                          train_data,
                          num_boost_round=best_iter)
    
    #iv. Predigo Agosto y obtengo las probabilidades.
    predicciones = model_lgb.predict(X_test)
    
    #v. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
    X_test["Probabilidad"] = predicciones
    
    #vi. Creamos un DataFrame con las probabilidades de la semilla.
    temp_df = X_test[["numero_de_cliente","foto_mes","Probabilidad"]]
    
    #vii. Renombramos.
    temp_df = temp_df.rename({"Probabilidad": f"prob_semilla_{semilla}"}, axis=1)
    
    #viii. Unimos los resultados al Dataframe general con todos los resultados.
    if resultados.empty:
        resultados = temp_df
    else:
        resultados = pd.merge(resultados, temp_df, on=['numero_de_cliente', 'foto_mes'], how='left')
        
    #ix. Borramos la columna con las Predicciones.
    X_test.drop(["Probabilidad"],axis=1,inplace=True)

In [ ]:
#5. Guardamos el dataframe para leerlo después.
resultados.to_csv(dataset_semillerio_all_1, index=False)
print("Archivo con semillerio guardado con éxito.")

In [ ]:
#6. Calculamos la probabilidad promedio de cada numero_de_cliente-foto_mes.
resultados["probabilidad_promedio"] = resultados.filter(like="prob_semilla_").mean(axis=1)

In [ ]:
#7. Filtro el dataframe para quedarme con el numero_de_cliente,foto_mes,probabilidad_promedio.
resultados_promedio = resultados[["numero_de_cliente","foto_mes","probabilidad_promedio"]]

In [ ]:
#8. Enviamos a Kaggle la probabilidad promedio por numero_de_cliente-foto_mes producto del semillerío.
#i. Activamos la API para realizar envíos a Kaggle.
api = KaggleApi()
api.authenticate()
#ii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
tb_entrega = resultados_promedio.sort_values(by='probabilidad_promedio', ascending=False)
#iii. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
cortes = range(10000,13000,250)
#iv.. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
num_subida_kaggle = 1
print("--> Comienzan los envíos a Kaggle...\n")
for envios in cortes:
    #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    #2. Guardamos el archivo para Kaggle.
    nombre_archivo = "K_semillerio_{}_{}_EK_0{}_{}.csv".format(cantidad_meses_train,ventana,num_subida_kaggle,semilla) #DF con -X meses, Y ventana, número de intento, número de semilla.
    ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
    resultados.to_csv(ruta_archivo, index=False)
    
    num_subida_kaggle += 1
    
    #3. Envío a Kaggle.
    #a. Defino los parámetros claves.
    mensaje = f'Archivo {nombre_archivo}.Train {cantidad_meses_train} semillerio. Semilla:{semilla},corte:{envios}' # Datos de Train, si está con Data Drifting o no, semilla y punto de corte de prueba.
    competencia = 'dm-ey-f-2024-tercera'
    #b. Subo la Submission.
    api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
    print("Submission successful!")